In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import mne
from mne.datasets import eegbci
from mne import Epochs, pick_types, find_events
import os
import scipy.io as sio

In [2]:
def read_dataset(dataset,num):
    path = "dataset"+str(dataset)+"/"
    if dataset == 1:
        path += "A"
    elif dataset == 2:
        path += "S"
    elif dataset == 4:
        path += "B"
    e = []
    t = []
    for i in range(1,num+1):
        e.append(np.asarray(sio.loadmat(path+str(i).zfill(2)+"E.mat")['data']))
        t.append(np.asarray(sio.loadmat(path+str(i).zfill(2)+"T.mat")['data']))
    return t, e

In [3]:
def prep_d1(data):
    dataset = []
    for i in range(len(data[0])):
        dataset.append(data[0][i])
    return np.array(dataset)

def prep_t4(data):
    data1 = list(data.tolist()[0][0][0][0])
    data2 = list(data.tolist()[0][1][0][0])
    data3 = list(data.tolist()[0][2][0][0])
    return np.array([data1, data2, data3])

def prep_e4(data):
    data1 = list(data.tolist()[0][0][0][0])
    data2 = list(data.tolist()[0][1][0][0])
    return np.array([data1, data2])


In [4]:
def features1(data):
    x = data[0][0][0][:,:22]
    trial_raw = data[0][0][1]
    trial = [num for e in trial_raw for num in e]
    y = [num for e in data[0][0][2] for num in e]
    atf = [num for e in data[0][0][5] for num in e]
    rate = data[0][0][3].tolist()
    
    return (x,trial,y,atf,rate[0][0])

def features4(data):
    x = data[0][:,:3]
    trial_raw = data[1]
    trial = [num for e in trial_raw for num in e]
    y = [num for e in data[2] for num in e]
    atf = [num for e in data[5] for num in e]
    rate = data[3].tolist()
    
    return (x,trial,y,atf,rate[0][0])

In [ ]:
def split_data(data):
    x, trial, y, atf, rate = data
    group1 = [[],[]]
    group2 = [[],[]]
    group3 = [[],[]]
    group4 = [[],[]]
    a = list(range(0, x.shape[0])) 
    
    z = list(zip(atf, y, trial))
    filtered = [(b,c) for a,b,c in z if a == 0]
    label, trial_filter = zip(*filtered)

    for i in a:
        if i in trial_filter:
            group1[0].append(x[i:i+2*rate])
            group2[0].append(x[i+int(2.5*rate):i+int(4.5*rate)])
            group3[0].append(x[i+4*rate:i+6*rate])
            group4[0].append(x[i+int(6.5*rate):i+int(8.5*rate)])
    
    group1[1] = [1] * len(group1[0])
    group2[1] = [2] * len(group2[0])
    group3[1] = [3] * len(group3[0])
    group4[1] = [4] * len(group4[0])
    
    arr1 = np.array(group1)
    arr2 = np.array(group2)
    arr3 = np.array(group3)
    arr4 = np.array(group4)
    
    #return np.concatenate((arr1,arr2))
    return arr1,arr2,arr3,arr4

In [6]:
t1, e1 = read_dataset(1,9)
t4, e4 = read_dataset(4,9)

In [7]:
data_t1 = [prep_d1(data) for data in t1]
data_e1 = [prep_d1(data) for data in e1]
data_t4 = [prep_t4(data) for data in t4]
data_e4 = [prep_e4(data) for data in e4]

In [8]:
x_t4 = [features4(data2) for data in data_t4 for data2 in data]
x_e4 = [features4(data2) for data in data_e4 for data2 in data]
x_t1 = []
for i in range(len(data_t1)):
    if i == 3:
        for j in range(1,len(data_t1[i])):
            x_t1.append(features1(data_t1[i][j]))
    else:
        for j in range(3,len(data_t1[i])):
            x_t1.append(features1(data_t1[i][j]))
            
x_e1 = []
for i in range(len(data_e1)):
    for j in range(3,len(data_e1[i])):
        x_e1.append(features1(data_e1[i][j]))

In [9]:
print('t1','*'*50)
for x in x_t1:
    print(len(x[0]),len(x[2]),x[4])
print('e1','*'*50)
for x in x_e1:
    print(len(x[0]),len(x[2]),x[4])
print('t4','*'*50)
for x in x_t4:
    print(len(x[0]),len(x[2]),x[4])
print('e4','*'*50)
for x in x_e4:
    print(len(x[0]),len(x[2]),x[4])

t1 **************************************************
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
e1 **************************************************
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48

In [ ]:
data1t = [split_data(x) for x in x_t1]
data1e = [split_data(x) for x in x_e1]
#data4t = [split_data(x) for x in x_t4]
#data4e = [split_data(x) for x in x_e4]

In [ ]:
def re_data(data):
    re_xt1 = []
    yt1 = []
    for x in data:
        for i, j in enumerate(x):
            if i%2 == 0:
                re_xt1.append(j)
            else:
                yt1.append(j)
    
    max_n = [d.shape[0] for d in re_xt1]
    max_n.sort(reverse=True)
    
    new_xt1 = np.zeros((len(re_xt1),max_n[0],re_xt1[0][0].shape[0],re_xt1[0][0].shape[1]))
    new_yt1 = np.zeros((len(yt1),max_n[0]))
    for i, a in enumerate(re_xt1):
        for j, b in enumerate(a):
            new_xt1[i][j] = b
        
    for i, a in enumerate(yt1):
        new_yt1[i,:a.shape[0]] = a
        
    return new_xt1, new_yt1

In [ ]:
x1t, y1t = re_data(data1t)
x1e, y1e = re_data(data1e)
x4t, y4t = re_data(data4t)
x4e, y4e = re_data(data4e)

In [ ]:
print("x:",x1t.shape,"y:",y1t.shape)
print("x:",x1e.shape,"y:",y1e.shape)
print("x:",x4t.shape,"y:",y4t.shape)
print("x:",x4e.shape,"y:",y4e.shape)

In [133]:
def write_npz_file(x,y,folder,head,subject):
    folder += head
    if not os.path.exists(folder):
        os.makedirs(folder)
    filename = str(folder)+"/S"+str(subject).zfill(3)+"_"+head+".npz"
    np.savez(filename,x=x,y=y)

In [134]:
x1t = x1t.reshape(9,-1,x1t.shape[1],x1t.shape[2],x1t.shape[3])
x1e = x1e.reshape(9,-1,x1e.shape[1],x1e.shape[2],x1e.shape[3])
x4t = x4t.reshape(9,-1,x4t.shape[1],x4t.shape[2],x4t.shape[3])
x4e = x4e.reshape(9,-1,x4e.shape[1],x4e.shape[2],x4e.shape[3])

y1t = y1t.reshape(9,-1,y1t.shape[1])
y1e = y1e.reshape(9,-1,y1e.shape[1])
y4t = y4t.reshape(9,-1,y4t.shape[1])
y4e = y4e.reshape(9,-1,y4e.shape[1])

print("x:",x1t.shape,"y:",y1t.shape)
print("x:",x1e.shape,"y:",y1e.shape)
print("x:",x4t.shape,"y:",y4t.shape)
print("x:",x4e.shape,"y:",y4e.shape)

x: (9, 12, 48, 500, 22) y: (9, 12, 48)
x: (9, 12, 48, 500, 22) y: (9, 12, 48)
x: (9, 6, 153, 500, 3) y: (9, 6, 153)
x: (9, 4, 156, 500, 3) y: (9, 4, 156)


In [135]:
for i, x in enumerate(x1t):
    write_npz_file(x,y1t[i],"lr_imagine_","1t",i+1)

for i, x in enumerate(x1e):
    write_npz_file(x,y1e[i],"lr_imagine_","1e",i+1)
    
for i, x in enumerate(x4t):
    write_npz_file(x,y4t[i],"lr_imagine_","4t",i+1)
    
for i, x in enumerate(x4e):
    write_npz_file(x,y4e[i],"lr_imagine_","4e",i+1)